From: https://w.wiki/6aJ3 updated https://w.wiki/6crp 

In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

from PIL import Image
import requests

endpoint_url = "https://query.wikidata.org/sparql"

query = """#authors with country of citizenship United States who do not have a Goodreads identifier
SELECT ?item ?itemLabel ?image WHERE {
  ?item wdt:P31 wd:Q5;
    wdt:P27 wd:Q30;
    wdt:P106 wd:Q36180.
  MINUS { ?item wdt:P2963 _:b1. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?item wdt:P18 ?image. }
}
LIMIT 9"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q14099'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Evi%20nemeth.jpg'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Evi Nemeth'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q14105'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Meredith%20L%20Patterson%2027C3.jpg'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Meredith L. Patterson'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q14536'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Seth%20Meyers%20by%20Gage%20Skidmore.jpg'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Seth Meyers'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15153'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Carl W. Ackerman'}}
{'item': {'type':